# Lab 4: Simulating a single qubit system
In this assignment, you’ll build a small (but real) quantum state simulator in Python while practicing core software engineering skills. You’ll implement a reusable `QubitSystem` interface (using an abstract base class) that supports initializing and inspecting quantum states and applying a handful of standard quantum gates, then create a concrete `SingleQubitSystem` implementation that inherits from it. Along the way, you’ll strengthen your understanding of how quantum states can be represented (bra-ket vs. state vectors), how **object-oriented** design helps you structure and extend code cleanly, and how test-driven development (TDD) can guide you to implement correct behavior incrementally with confidence.


# Object-oriented programming (OOP)

Object-oriented programming is a way to organize code around “objects” that bundle **data** (attributes) with **behavior** (methods). A core OOP idea is to define a general blueprint for a family of related things, then create more specific versions by extending that blueprint. This helps you reuse code, keep responsibilities clear, and make it easier to add new features without rewriting existing logic.

### Base classes and shared behavior

A common pattern is to create a **base class** (sometimes called an interface) that defines what all related objects should be able to do. In the example below, `Polygon` represents “any shape with sides.” Because every polygon has a number of sides, we can implement `print_number_of_sides()` once in `Polygon` and automatically reuse it in every child class.

### Abstract methods and specialization

Some behaviors depend on the specific type of polygon. For instance, the formula for area is different for triangles and rectangles, so `calculate_area()` is declared as an abstract method in `Polygon`. That means `Polygon` is promising “every polygon can calculate an area,” but it requires each subclass to provide the actual formula.

### Inheritance and `super()`

When you write a child class, you often still want to run some of the “standard setup” code that the parent class already provides. Python’s `super()` lets you call that parent-class code from inside your child class, instead of copying it.

For example, in `Triangle`, we call `super().__init__(num_sides=3)` to run the `Polygon` constructor so the triangle is initialized with the correct number of sides. Then we add triangle-specific fields like `base` and `height`.


In [ ]:
from abc import ABC, abstractmethod

class Polygon(ABC):
    def __init__(self, num_sides: int):
        self.num_sides = num_sides

    def print_number_of_sides(self) -> str:
        return f"This polygon has {self.num_sides-1} sides."

    @abstractmethod
    def calculate_area(self) -> float:
        """Return the area of this polygon."""
        raise NotImplementedError


class Triangle(Polygon):
    def __init__(self, base: float, height: float):
        super().__init__(num_sides=3)
        self.base = base
        self.height = height

    def calculate_area(self) -> float:
        return 0.5 * self.base * self.height


class Rectangle(Polygon):
    def __init__(self, width: float, height: float):
        super().__init__(num_sides=4)
        self.width = width
        self.height = height

    def calculate_area(self) -> float:
        return self.width * self.height


In [ ]:
# We can now instantiate (create an instance of) a triangle:
t = Triangle(10, 4)

# Compute its area
print(t.calculate_area())

# We can also instantiate a rectangle:
r = Rectangle(10, 4)

# It has a different area!
print(r.calculate_area())

In this assignment, you’ll use the same design approach: define a general (abstract) `QubitSystem` with shared behaviors where possible, then implement a specific `SingleQubitSystem` that inherits from it and fills in the details (or restricts operations that don’t make sense).


# Test Driven Development

Test-driven development (TDD) means you write a small test first, watch it fail (because the feature isn’t implemented yet), then write the minimum code to make it pass, and finally clean up your design without changing behavior. This workflow helps you lock in expected behavior early and catch regressions as you refactor your classes.

The general approach for a test is:
- **Arrange**: create the instances of the object you are testing
- **Act**: perform actions or operations on the instance of the object
- **Assert**: validate the result of the actions being tested

### What we’re testing

For the `Triangle` subclass, we’ll write two tests:

- `test_calculate_area`: verifies `calculate_area()` returns the correct numeric result for known inputs.
- `test_print_number_of_sides`: verifies `print_number_of_sides()` prints the expected text (since it’s implemented in the parent `Polygon` class but should work when called on a `Triangle`).

Each test will return `True` if behavior is correct and `False` otherwise, so you can run them without any testing framework.

### Test 1: `test_calculate_area`

This test constructs a triangle with a known base and height, then checks whether the computed area matches what you expect.


In [ ]:
def test_calculate_area() -> bool:
    # Arrange
    t = Triangle(base=10, height=4)

    # Act
    area = t.calculate_area()

    # Assert
    expected = 20.0  # 0.5 * 10 * 4
    return area == expected


Now let's write a test for `print_number_of_sides`.

In [ ]:
def test_print_number_of_sides() -> bool:
    # Arrange
    t = Triangle(base=3, height=5)

    # Act
    printed = t.print_number_of_sides()

    # Assert
    return printed == "This polygon has 3 sides."

### Running your tests

A simple runner can quickly call each test and print whether it passed:

In [ ]:
def run_tests() -> None:
    tests = [
        ("test_calculate_area", test_calculate_area),
        ("test_print_number_of_sides", test_print_number_of_sides),
    ]

    for name, fn in tests:
        result = fn()
        print(f"{name}: {'PASS' if result else 'FAIL'}")

run_tests()


Oh no! The test for number of sides failed! That means our code must have a bug in it.

## Task 0: Ensure our tests pass

Find and fix the bug in our code above, then run the test suite again to make sure it passes!

# Assignment Specs:
Now that you are familiar with OOP we can move on the `QubitSystem`. You will be implementing the following:


### `QubitSystem` technical specs

| Function | Behavior |
| --- | --- |
| `set_value(value)` | Sets the system’s quantum state. Accept a list of floats representing the state vector amplitudes in **big-endian** basis order. Implementations should validate that the input represents a valid state for the system size and raise a `ValueError` if not. |
| `get_value_braket() -> str` | Returns a bra-ket style string representing the current state. This is primarily for readability/debugging, you won't be penalized for stylistic differences or formating. |
| `get_value_vector() -> list[float]` | Returns the current state vector amplitudes in big-endian basis order. |
| `apply_not(i: int) -> None` | Applies the NOT gate (Pauli-$X$) to qubit at index $i$, updating the system state. Raise an `IndexError` if $i$ is not a valid qubit index. |
| `apply_h(i: int) -> None` | Applies the Hadamard gate ($H$) to qubit at index $i$, updating the system state. Raise an `IndexError` if $i$ is not a valid qubit index. |
| `apply_z(i: int) -> None` | Applies the Pauli-$Z$ gate to qubit at index $i$, updating the system state. Raise an `IndexError` if $i$ is not a valid qubit index. |
| `apply_cnot(control: int, target: int) -> None` | Applies a controlled-NOT gate with `control` as the control qubit and `target` as the target qubit, updating the system state. Raise an `IndexError` if either index is invalid, or if they are the same. |
| `apply_swap(i: int, j: int) -> None` | Swaps qubits at indices $i$ and $j$, updating the system state. Raise an `IndexError` if either index is invalid, or if they are the same. |
| `measure() -> str` | <p>Simulates a measurement of the state of the system and returns one of the possible values as a big-endian string of binary: e.g., if the state $\lvert 00 \rangle$ is being measured the result would always be `'00'`, If the state $\frac{1}{\sqrt{2}} \lvert 00 \rangle + \frac{1}{\sqrt{2}} \lvert 01 \rangle$ is measured half the time we would expect `'00'` the other half we would expect `'01'`.<ul><li>Note: If a system is in a state of superposition before `measure()`, the act of measurement should collapse the superposition.</li><li>Note 2: The output should always have the same number of bits as the system does. For `SingleQubitSystem` the outputs will be `'0'` or `'1'`.</li></ul></p> |

### `SingleQubitSystem` technical specs

`SingleQubitSystem`: A concrete `QubitSystem` implementation for exactly one qubit. It must support initialization, retrieval in both formats, and single-qubit gates (`apply_not`, `apply_h`, `apply_z`) with correct state updates.

| Function | Behavior |
| --- | --- |
| `set_value(value)` | In addition to supporting a list of floats, the `SingleQubitSystem`'s `set_value` will also accept the following strings representing bra-ket states: `"\|0>"`, `"\|1>"`, `"\|+>"` or `"\|->"` |
| `apply_cnot(control: int, target: int) -> None` | Must always raise an `IndexError` when called on a single-qubit system (since there is no valid pair of qubit indices). |
| `apply_swap(i: int, j: int) -> None` | Must always raise an `IndexError` when called on a single-qubit system (since there is no valid pair of qubit indices). |


## Importing external libraries (don't): 
You are free to use [`numpy`](https://numpy.org/doc/stable/) to handle all matrix and mathematical operations or you may choose to implement everything yourself as helper functions. Regardless of your decision, we expect all students to know how to programmatically implement matrix multiplication (*hint: potential midterm question*). You may **not** use `qiskit` or **any** other library besides `numpy` as part of your solution.
**Adding extra import statement _will_ crash the autograder**

## Implementation Steps:
For now *all* of `QubitSystem`'s functions are marked as abstract; it is up to you to decide which ones to implement and which ones to delegate (i.e., only implement in child classes).

If you decide the functionality is broad enough that ALL qubit systems regardless of number of qubits would be able to use it you should:


1.   Implement it in `QubitSystem`
2.   Remove the `@abstractmethod` tag
3.   Delete the function definition from `SingleQubitSystem` (unless you will be [overloading](https://en.wikipedia.org/wiki/Function_overloading) it. For an example of this see the `__init__` methods)

If instead you think a particular set of functionality should be implemented by `SingleQubitSystem` you may leave everything as is and go straight to `SingleQubitSystem`.

Next week's lab will involve implementing an `NQubitSystem` which *will* be able to handle all gates so it is worth the effort to plan out a versatile implementation **now** rather than having to deal with refractoring later.

Note: if you decide to declare helper functions **make sure to include them within your class declarations**; otherwise, your submission might crash when being autograded.

For your convenience, we have provided skeleton code to get you started. **Do not rename any of the provided functions.**

## Task 1: Before you start implementing, scroll down and review the test suite

In [ ]:
from abc import ABC, abstractmethod
import random

class QubitSystem(ABC):
    """
    Abstract interface for a system of qubits.

    Child classes should store and update the quantum state, and must raise
    IndexError when gate indices are out of bounds.
    """

    def __init__(self, num_qubits: int):
        self._num_qubits = num_qubits
        self.state = [0]*2**num_qubits
        self.state[0] = 1

    @property
    def num_qubits(self) -> int:
        return self._num_qubits

    def _check_index(self, i: int) -> None:
        """Helper for consistent index-out-of-bounds behavior."""
        if i < 0 or i >= self._num_qubits:
            raise IndexError(f"Qubit index out of bounds: {i}")

    def _check_pair(self, i: int, j: int) -> None:
        """Helper for two-qubit operations."""
        self._check_index(i)
        self._check_index(j)

    @abstractmethod
    def set_value(self, value) -> None:
        """Set the current state using bra-ket string or state-vector list."""
        raise NotImplementedError

    @abstractmethod
    def get_value_braket(self) -> str:
        """Return a bra-ket formatted string for the current state."""
        raise NotImplementedError

    @abstractmethod
    def get_value_vector(self) -> list[float]:
        """Return the state vector (big-endian basis ordering)."""
        raise NotImplementedError

    @abstractmethod
    def apply_not(self, i: int) -> None:
        """Apply NOT (Pauli-X) to qubit i."""
        raise NotImplementedError

    @abstractmethod
    def apply_h(self, i: int) -> None:
        """Apply Hadamard to qubit i."""
        raise NotImplementedError

    @abstractmethod
    def apply_z(self, i: int) -> None:
        """Apply Pauli-Z to qubit i."""
        raise NotImplementedError

    @abstractmethod
    def apply_cnot(self, control: int, target: int) -> None:
        """Apply CNOT with given control and target qubits."""
        raise NotImplementedError

    @abstractmethod
    def apply_swap(self, i: int, j: int) -> None:
        """Swap qubits i and j."""
        raise NotImplementedError

    @abstractmethod
    def measure(self) -> str:
        """Simualte a measurement of the system."""
        raise NotImplementedError

class SingleQubitSystem(QubitSystem):
    def __init__(self):
        super().__init__(num_qubits=1)

    def set_value(self, value) -> None:
        raise NotImplementedError

    def get_value_braket(self) -> str:
        raise NotImplementedError

    def get_value_vector(self) -> list[float]:
        raise NotImplementedError

    def apply_not(self, i: int) -> None:
        raise NotImplementedError

    def apply_h(self, i: int) -> None:
        raise NotImplementedError

    def apply_z(self, i: int) -> None:
        raise NotImplementedError

    def apply_cnot(self, control: int, target: int) -> None:
        raise NotImplementedError

    def apply_swap(self, i: int, j: int) -> None:
        raise NotImplementedError

    def measure(self) -> str:
        raise NotImplementedError

# Test Suite
We will begin by defining the bare-bones testing suite. **The autograder will expect for these tests to be present in your submission and will evaluate them for correctness; do not change their names.**

As you progress through the lab be sure to revisit your tests and come up with meaningful ways to evaluate your program's functionality under known circumstances. Your tests should always exit gracefully, even when intentionally triggering errors (e.g., ensuring a `SingleQubitSystem` throws an `IndexError` when a multi-qubit gate is applied): your code should catch the errors rather than letting them propagate.

You will be graded on your test-suite's coverage in addition to the correctness of your `QubitSystem` and `SingleQubitSystem` classes.

## Task 2: Comparing `float`s
Comparing floating point numbers is trickier than it might seem due to rounding errors. The following exmple shows how even mathematically identical numbers can be interpreted as different due to the way computers handle data:


In [ ]:
a = 0.0
b = 0.0

# Same math, different grouping
for _ in range(1_000_000):
    a += 0.1

b = 0.1 * 1_000_000

print(a)
print(b)
print(a == b)

To help with this, we have provided with a `compare_lists` function. It compares two lists and, so long as all numbers are within a small margin of each other (conventionally labeled as epsilon $\epsilon$), it will return `True`. We recommend you use it when testing your code.

In [ ]:
def compare_lists(first_list, second_list, eps: float = 1e-3) -> bool:
    import numpy as np
    return bool(np.allclose(first_list, second_list, atol=eps, rtol=0.0))

print([a]==[b])
print(compare_lists([a], [b]))

# Task 3: Implement `set_value`
We've provided a minimal test case for `set_value`. Scroll up to the class definitions, implement the barebones functionality (storing a properly formatted list) required to pass the test, then run the cells containing your class definitions and testing suite again.

The autograder will evaluate each of the test cases below for coverage so you must add multiple cycles of arrange-act-assert within each case. **Do not rename these test cases, any helper functions must be defined _within_ an existing test case**

In [ ]:
def test_set_value() -> bool:
    # Arrange: Set up a new SingleQubit to evaluate
    q = SingleQubitSystem()

    # Act: Evaluate the code you are testing
    q.set_value([1.0, 0.0])

    # Assert: Compare the results with the expected outcome

    return compare_lists(q.state, [1.0,0.0])


def test_get_value_vector() -> bool:
    # Arrange: Set up a new SingleQubit to evaluate

    # Act: Evaluate the code you are testing

    # Assert: Compare the results with the expected outcome

    return False  # overwrite when ready


def test_apply_not() -> bool:
    # Arrange: Set up a new SingleQubit to evaluate

    # Act: Evaluate the code you are testing

    # Assert: Compare the results with the expected outcome

    return False  # overwrite when ready


def test_apply_h() -> bool:
    # Arrange: Set up a new SingleQubit to evaluate

    # Act: Evaluate the code you are testing

    # Assert: Compare the results with the expected outcome

    return False  # overwrite when ready


def test_apply_z() -> bool:
    # Arrange: Set up a new SingleQubit to evaluate

    # Act: Evaluate the code you are testing

    # Assert: Compare the results with the expected outcome

    return False  # overwrite when ready


def test_apply_cnot() -> bool:
    # Arrange: Set up a new SingleQubit to evaluate

    # Act: Evaluate the code you are testing

    # Assert: Compare the results with the expected outcome

    return False  # overwrite when ready


def test_apply_swap() -> bool:
    # Arrange: Set up a new SingleQubit to evaluate

    # Act: Evaluate the code you are testing

    # Assert: Compare the results with the expected outcome

    return False  # overwrite when ready


def test_measure() -> bool:
    # Arrange: Set up a new SingleQubit to evaluate

    # Act: Evaluate the code you are testing
    # hint: How do you test non-deterministic behavior? Is once enough?

    # Assert: Compare the results with the expected outcome

    return False  # overwrite when ready




def run_tests() -> None:
    tests = [
        ("test_set_value", test_set_value),
        ("test_get_value_vector", test_get_value_vector),
        ("test_apply_not", test_apply_not),
        ("test_apply_h", test_apply_h),
        ("test_apply_z", test_apply_z),
        ("test_apply_cnot", test_apply_cnot),
        ("test_apply_swap", test_apply_swap),
        ("test_measure", test_measure),
    ]

    for name, fn in tests:
        try:
            result = fn()
        except Exception as e:
            print(f'Exception on {name}:', e)
            result = False

        print(f"{name}: {'PASS' if result else 'FAIL'}")

run_tests()

## Task 4: Implement everything else!
Remember to be thorough with your test cases! For example, even though our `set_value` test is now non-trivial, it is far from exhaustive; it does not test for:


1.   String inputs: Per the specs SingleQubit should support the following strings representing bra-ket states: `"|0>"`, `"|1>"`, `"|+>"` and `"|->"`
2.   States with an incorrect number of qubits
3.   Invalid states

Good luck!

# Submission
Congratulations on completing the lab!
Make sure you:


1.   Test all of your functions by calling them at least once.
2.   Download your lab as a **python** `.py` script (NOT an `.ipynb` file):
      
      ```File -> Download -> Download .py```

3.   Rename the downloaded file to `Lab4Answers.py`.
4.   Upload the `Lab4Answers.py` file to Gradescope.
5.   Ensure the autograder runs successfully.